## Evaluation of Knowledge Graph for RAG

In [1]:
from database.GraphDB import KnowledgeGraphDB
from retrieval.retrieval_hub import RetrievalHub
from query_and_ranking.query import Query
from query_and_ranking.rerank import Reranker
from agentic import Agent
from tqdm import tqdm

2024-06-10 01:45:51.974210: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 01:45:51.974232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 01:45:51.974825: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 01:45:52.541908: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

neo4j_uri = os.getenv('NEO4J_URI')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')

In [3]:
kg = KnowledgeGraphDB(uri=neo4j_uri, user=neo4j_user, password=neo4j_password)
retrieval_hub = RetrievalHub(kg)
reranker = Reranker()
cv_query = Query(kg, retrieval_hub)

In [4]:
chat_agent = Agent(kg, retrieval_hub, reranker, cv_query, routing='aws',jd_extraction='local')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
import json
questions = json.loads(open('data/benchmark/qa_cv.json').read())

In [6]:
answers = []
i = 0

for question in tqdm(questions):
    prompt = question['prompt']
    answer = dict()
    answer['prompt'] = prompt
    answer['answer'] = question['answer']
    try:
        res = chat_agent.get_cv_from_jd(prompt)
        ans = res['files']
        answer['predict'] = [a.replace('data/raw/data/clean4.0/', '') for a in ans]
    except:
        print('Error')
        answer['predict'] = []
    answers.append(answer)
    with open('data/benchmark/qa_cv_kg.json', 'w') as f:
        f.write(json.dumps(answers))
    i += 1


  0%|          | 0/39 [00:00<?, ?it/s]

Extracting Job Description
End LLM
New Query:  {'cityList': ['ha noi'], 'eduList': ['master', 'bachelor', 'undergraduate', 'diploma', 'phd', 'certificate'], 'roleList': ['marketing analyst', 'marketing officer', 'marketing manager', 'marketing assistant'], 'suitableList': ['marketing manager', 'marketing executive', 'director of marketing', 'marketing officer', 'head of marketing', 'marketing team leader']}
Extended Num Application:  15


/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Languages:\n- English\nMajors:\n- bachelor, business administration - marketing, fpt university, (GPA: 3.2)\nWork:\n- nha khoa parkway (1 years)\n- tổ chức giáo dục fpt (0.5 years)\n- trung tâm tin học ict hà nội (1 years)\n- công ty cổ phân đầu tư và phát triển hạ tầng việt nam (1.5 years)\n- sconnect vietnam (1.5 years)\nRoles:\n- content marketing (1.5 years)\n- digital marketing (1 years)\n- social media manager (1.5 years)\n- marketing executive (1.5 years)\nSkills:\n- market research\n- content creation\n- social media marketing\n- digital marketing\n- advertising campaign management\n- customer journey optimization\n- team management\n- seo\nCities:\n- ha noi\n', metadata={'id': 1749}), -225.46678579783702), (Document(page_content='Languages:\n- English\nMajors:\n- bachelor, information technology, hano

Not enough application
New Query:  {'cityList': ['ha noi'], 'suitableList': ['marketing staff', 'marketing manager', 'marketing executive', 'director of marketing', 'marketing officer', 'head of marketing', 'marketing', 'marketing team leader']}
[786, 1749, 1802, 2109, 424]
Less -> Sematic Search
Using Sematic Search
Num CV ok:  12
Rerank
[1513, 1593, 1749, 1802, 2006, 2109, 222, 395, 424, 595, 786, 841]
Time: 12.240367412567139
Extracting Job Description


  5%|▌         | 2/39 [00:23<07:08, 11.59s/it]

...

End LLM
New Query:  {'institutionList': ['ho chi minh city university of technology and food industry', 'ho chi minh city university of industry and trade'], 'eduList': ['master', 'bachelor', 'undergraduate', 'diploma', 'phd', 'certificate']}
Extended Num Application:  3
Num CV ok:  3
Rerank
Reranking
[1841, 1066]
Time: 9.401374578475952
Extracting Job Description


/home/quanghung20gg/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Languages:\n- English\nMajors:\n- bachelor, information security, ho chi minh city university of technology and food industry, (GPA: 2.9)\nWork:\n- công ty c ổ  ph ầ n công ngh ệ tam khoadeveloper .net (1 years)\nRoles:\n- tester (0 years)\n- researcher (0.5 years)\n- software developer (1 years)\nSkills:\n- software development\n- machine learning\n- sql\n- technical support\nProgramming languages:\n- .net (1 years)\n- python (0.5 years)\n- c# (1 years)\n- sql (1 years)\nCities:\n- ho chi minh\n', metadata={'id': 1841}), -216.83886600124606), (Document(page_content='Languages:\n- English\nMajors:\n- bachelor, information technology, vnu-hcm university of science, (GPA: 3.6)\nRoles:\n- software engineer (0.5 years)\n- web developer (0.5 years)\nSkills:\n- cloud storage\n- devops\n- machine learning\n- database

End LLM
New Query:  {'institutionList': ['national economics university hanoi', 'ho chi minh city university of economics'], 'eduList': ['master', 'bachelor', 'undergraduate', 'diploma', 'phd', 'certificate']}
Extended Num Application:  11
Not enough application
New Query:  {'institutionList': ['national economics university hanoi', 'ho chi minh city university of economics']}
[1593, 669, 468, 135, 1958]
Less -> Sematic Search
Using Sematic Search
Num CV ok:  8
Rerank
[1240, 135, 1488, 1593, 1841, 1958, 468, 669]
Time: 10.017922163009644
Extracting Job Description
End LLM
New Query:  {'institutionList': ['hanoi university of science and technology', 'hanoi university of business and technology', 'university of technology information - vnu', 'posts and telecommunications institute of technology', 'vnu-hcm university of science'], 'eduList': ['master', 'bachelor', 'undergraduate', 'diploma', 'phd', 'certificate']}
Extended Num Application:  12
Num CV ok:  10
Rerank
Reranking


100%|██████████| 39/39 [06:15<00:00,  9.64s/it]

[751, 345, 222, 2083, 1104, 19, 1733, 1488, 2051]
Time: 10.418983221054077


In [10]:
recall = 0
precision = 0
long_recall = 0
long_precision = 0
len_long = 0
short_recall = 0
short_precision = 0
len_short = 0

for answer in answers:
    num_ans = len(answer['answer'])
    correct = 0
    ans = set(answer['answer'])
    for p in answer['predict']:
        if p in ans:
            correct += 1
    recall += correct / num_ans
    precision += correct / (len(answer['predict'])+1e-5)
    if num_ans > 7:
        long_recall += correct / num_ans
        long_precision += correct / (len(answer['predict'])+1e-5)
        len_long+= 1
    else:
        short_recall += correct / num_ans
        short_precision += correct / (len(answer['predict'])+1e-5)
        len_short += 1

In [11]:
recall /= len(answers)
precision /= len(answers)
long_recall /= len_long
long_precision /= len_long
short_recall /= len_short
short_precision /= len_short
f1 = 2 * recall * precision / (recall + precision + 1e-5)
short_f1 = 2 * short_recall * short_precision / (short_recall + short_precision + 1e-5)
long_f1 = 2 * long_recall * long_precision / (long_recall + long_precision+1e-5)

In [12]:
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1: {f1}")
print("=====================================")

print(f"Short Recall: {short_recall}")
print(f"Short Precision: {short_precision}")
print(f"Short F1: {short_f1}")
print("=====================================")

print(f"Long Recall: {long_recall}")
print(f"Long Precision: {long_precision}")
print(f"Long F1: {long_f1}")
print("=====================================")

Recall: 0.4816278166278167
Precision: 0.4387250300532934
F1: 0.4591714650075473
Short Recall: 0.4360215053763441
Short Precision: 0.39011169349691877
Short F1: 0.41178596663490963
Long Recall: 0.6583522727272727
Long Precision: 0.6271017092092451
Long F1: 0.6423421290530603


## Haiku

Recall: 0.4582659932659932 \
Precision: 0.45441863442709507 \
F1: 0.45632920485280953

=====================================

Short Recall: 0.4209677419354839 \
Short Precision: 0.42419216192259934 \
Short F1: 0.4225688012224169

=====================================

Long Recall: 0.6027967171717171 \
Long Precision: 0.5715462153820161 \
Long F1: 0.5867506654834951

=====================================

## Llama 3 8b

Recall: 0.4816278166278167 \
Precision: 0.4387250300532934 \
F1: 0.4591714650075473

=====================================

Short Recall: 0.4360215053763441 \
Short Precision: 0.39011169349691877 \
Short F1: 0.41178596663490963

=====================================

Long Recall: 0.6583522727272727 \
Long Precision: 0.6271017092092451 \
Long F1: 0.6423421290530603 

=====================================

## Llama 3 70b